https://jira.x5.ru/browse/CVMXC-2632

In [1]:
name = 'CVMXC-2632_nestle_pa'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/04 16:10:59 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


### X-28-38-Nestle-Purina-Friskies-5days-sof

In [5]:
campaign_id = 'X-28-38-Nestle-Purina-Friskies-5days-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        80401: "sms_viber_tg",
        80401: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-21',
    'date_communication_to': '2022-01-27',

    'wave_ids': [
        80512 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-21',
    'date_to': '2022-01-27', #дата постпериода
    'control_camp_wave_id': 80401,
    'plu_list': list(map(str, [4125613, 4125614])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [6]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [7]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [8]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [9]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
1_80401_sms_viber_cg,16000,"-8,605.17",0.00,0.00,0.00,0.00,"408,772.75","43,819.00",31.00,13813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0_80401_sms_viber_cg,64997,"29,222.67",1.36,"88,573.98",1.39,"90,144.00","1,644,172.33","177,917.00",337.00,55912,...,"-252,485.00",-0.01,0.00,0.33,"21,688.00",0.19,0.00,0.46,"29,636.00",0.48


In [10]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### X-28-39-Nestle-Purina-Friskies-10days-sof

In [11]:
campaign_id = 'X-28-39-Nestle-Purina-Friskies-10days-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        80410: "sms_viber_tg",
        80410: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-21',
    'date_communication_to': '2022-01-27',

    'wave_ids': [
        80516 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-21',
    'date_to': '2022-01-27', #дата постпериода
    'control_camp_wave_id': 80410,
    'plu_list': list(map(str, [4125613, 4125614])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [12]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [13]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [14]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [15]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_80410_sms_viber_cg,64992,"46,001.56",1.36,"88,566.48",1.39,"90,040.50","1,661,415.55","183,048.00",222.00,56218,...,"-231,200.00",-0.01,0.00,-0.18,"-11,979.00",-0.10,0.00,-0.51,"-32,792.00",-0.74
1_80410_sms_viber_cg,16000,"-19,905.51",0.00,0.00,0.00,0.00,"407,116.67","43,660.00",32.00,13846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### U-28-Purina-Friskies-21.01-postavschik

In [17]:
campaign_id = 'U-28-Purina-Friskies-21.01-postavschik'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        80359: "sms_viber_tg",
        80380: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-21',
    'date_communication_to': '2022-01-27',

    'wave_ids': [
        80483 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-21',
    'date_to': '2022-01-27', #дата постпериода
    'control_camp_wave_id': 80380,
    'plu_list': list(map(str, [4125613, 4125614])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [18]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [19]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [20]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [21]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_80359_sms_viber_tg,254819,"6,220.63",1.36,"346,182.96",1.45,"370,221.00","5,873,463.89","639,283.00","1,896.00",215477,...,"-5,669,820.00",-0.07,0.00,0.06,"15,304.00",0.02,0.00,0.20,"49,881.00",0.14
0_80380_sms_viber_cg,46232,"-59,869.22",0.00,0.00,0.00,0.00,"1,117,576.06","118,331.00",221.00,39223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

22/02/04 23:03:58 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
22/02/04 23:03:58 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Attempt recovered after RM restartDue to executor failures all available nodes are blacklisted
22/02/04 23:03:58 ERROR client.TransportClient: Failed to send RPC RPC 6330602615737046249 to /192.168.234.56:32998: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/04 23:03:58 ERROR cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(0,0,Map(),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 6330602615737046249 to /192.168.234.56:32998: java.nio.channels.ClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.jav